In [ ]:
# Set up packages for lecture. Don't worry about understanding this code, but
# make sure to run it if you're following along.
import numpy as np
import babypandas as bpd
import pandas as pd
from matplotlib_inline.backend_inline import set_matplotlib_formats
import matplotlib.pyplot as plt
from scipy import stats
import otter
set_matplotlib_formats("svg")
plt.style.use('ggplot')

import warnings
warnings.simplefilter('ignore')

np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option("display.max_rows", 7)
pd.set_option("display.max_columns", 8)
pd.set_option("display.precision", 2)

# Animation
from IPython.display import IFrame, display

def show_clt_slides():
    src = "https://docs.google.com/presentation/d/e/2PACX-1vTcJd3U1H1KoXqBFcWGKFUPjZbeW4oiNZZLCFY8jqvSDsl4L1rRTg7980nPs1TGCAecYKUZxH5MZIBh/embed?start=false&loop=false&delayms=3000"
    width = 960
    height = 509
    display(IFrame(src, width, height))

# Lecture 26 – Review
## DSC 10, Winter 2023

### Announcements

- The Final Exam is **this Saturday 3/18 from 3-6PM in Galbraith Hall 242**.
    - See [this EdStem announcement](https://edstem.org/us/courses/33310/discussion/2784201) for details.
    - Assigned seats will be emailed to you by Friday.
    - We will check IDs.
    - You'll have **2 hours, 50 minutes** to work on the exam.
    - No questions during the exam.
    - The [DSC 10 Reference Sheet](https://drive.google.com/file/d/1ky0Np67HS2O4LO913P-ing97SJG0j27n/view) will be provided. No calculators or other notes.
    - Practice with old exam problems at [practice.dsc10.com](https://practice.dsc10.com).
- If at least 80% of the class fills out both [CAPEs](https://cape.ucsd.edu) and the [End of Quarter Survey](https://docs.google.com/forms/d/e/1FAIpQLSc7qjCK0DkBThgKw9SdNY9OX44SFA-UHX3MZ42dfEMcssCnWA/viewform), then the entire class gets 0.5% of extra credit on their overall grade.
    - We value your feedback!

### Agenda

- No new material – just review!
- If you're attending lecture, fill in the code in the notebook as we go. We'll post the solutions later today.

## The data: Restaurants 🍟

Our data comes from [data.sfgov.org](https://data.sfgov.org/Health-and-Social-Services/Restaurant-Scores-LIVES-Standard/pyih-qa8i/data).

In [ ]:
restaurants = bpd.read_csv('data/restaurants.csv')
restaurants

We won't look at many of the columns in our DataFrame, so let's just `get` the ones we're interested in.

In [ ]:
keep_cols = ['business_name', 'inspection_date', 'inspection_score', 'risk_category', 'Neighborhoods', 'Zip Codes']
restaurants = ...
restaurants

## At-risk restaurants ⚠️

For each restaurant, we have an inspection score.

In [ ]:
restaurants.take(np.arange(5))

In the preview above, we see...
- A restaurant with an inspection score of 92 being classified as `'Low Risk'`,
- A restaurant with an inspection score of 91 being classified as `'High Risk'`
- A restaurant with an inspection score of 90 being classified as `'Low Risk'`

This means that inspection scores don't directly translate to risk categories. Let's investigate the difference between the inspection scores of low risk and high risk restaurants.

Let's start by visualizing the distribution of inspection scores for low risk and high risk restaurants.

In [ ]:
# Don't worry about the plotting code here.
fig, ax = plt.subplots()
score_bins = np.arange(50, 102, 2)
restaurants[restaurants.get('risk_category') == 'Low Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), title='Inspection Scores for Low Risk vs. High Risk Restaurants', alpha=0.65, label='Low Risk'
);

restaurants[restaurants.get('risk_category') == 'High Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), alpha=0.65, label='High Risk'
);

### Concept Check ✅ – Answer at [cc.dsc10.com](http://cc.dsc10.com) 

We want to compare low risk restaurants to high risk restaurants and see if their inspection scores are significantly different. What technique should we use?

A. Standard hypothesis testing

B. Permutation testing  

C. Bootstrapping

D. The Central Limit Theorem

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
Permutation testing.

</details>

Let's keep only the relevant information.

In [ ]:
high_low = ...
high_low = ...
high_low

Now, let's try shuffling a single one of the columns above. (Does it matter which one?)

In [ ]:
...

Let's assign this shuffled column back into our original DataFrame. The resulting DataFrame is called `original_and_shuffled`.

In [ ]:
shuffled_labels = ...
# add a new column called shuffled_label
original_and_shuffled = ...
original_and_shuffled

Let's now visualize the distribution of inspection scores for low risk and high risk restaurants, in both our original dataset and after shuffling the labels.

In [ ]:
# Don't worry about the plotting code here.
fig, ax = plt.subplots()
score_bins = np.arange(50, 102, 2)
restaurants[restaurants.get('risk_category') == 'Low Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), title='Inspection Scores for Low Risk vs. High Risk Restaurants Before Shuffling', alpha=0.65, label='Low Risk'
);

restaurants[restaurants.get('risk_category') == 'High Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), alpha=0.65, label='High Risk'
);

In [ ]:
# Don't worry about the plotting code here.
fig, ax = plt.subplots()
score_bins = np.arange(50, 102, 2)
original_and_shuffled[original_and_shuffled.get('shuffled_label') == 'Low Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), title='Inspection Scores for Low Risk vs. High Risk Restaurants After Shuffling', alpha=0.65, label='Low Risk'
);

original_and_shuffled[original_and_shuffled.get('shuffled_label') == 'High Risk'].plot(
    kind='hist', y='inspection_score', density=True, ec='w', bins=score_bins, ax=ax,
    figsize=(10, 5), alpha=0.65, label='High Risk'
);

### Concept Check ✅ – Answer at [cc.dsc10.com](http://cc.dsc10.com) 

It looks like the two groups in the first histogram are substantially more different than the two groups in the second histogram. 

What test statistic(s) can we use to quantify the difference between the two groups displayed in a given histogram?

A. Total variation distance  
B. Difference in group means  
C. Either of the above

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
Difference in group means. TVD helps compare two categorical distributions, but we're dealing with two numerical distributions.

</details>

In [ ]:
...

Let's compute the difference in mean inspection scores for the low risk group and high risk group (low minus high).

First, for our observed data:

In [ ]:
grouped = ...
observed_difference = ...
observed_difference

Then, for our shuffled data:

In [ ]:
...

In [ ]:
shuffled_and_grouped = ...
simulated_difference = ...
simulated_difference

We're going to need to shuffle the `'risk_category'` column many, many times, and compute this difference in group means each time.

Let's put some of our code in a function to make it easier to repeat.

In [ ]:
def calculate_test_statistic():
    ...

Each time we call this function, it shuffles the `'risk_category'` column and returns the difference in group means (again, by taking low minus high).

In [ ]:
calculate_test_statistic()

We need to simulate this difference in group means many, many times. Let's call our function many, many times and keep track of its result in an array.

In [ ]:
simulated_stats = np.array([])
n_reps = 100 # We're using a small number of reps to save time in lecture.

...

Now that we've done that, let's visualize the distribution of the simulated test statistics, and also see where the observed statistic lies:

In [ ]:
bpd.DataFrame().assign(simulated_stats=simulated_stats) \
               .plot(kind='hist', density=True, ec='w', figsize=(10, 5), bins=20, label='difference in group means');
plt.axvline(observed_difference, lw=3, color='black', label='observed statistic')
plt.legend();

What's the p-value? Well, it depends on what our alternative hypothesis is. Here, our alternative hypothesis is that low risk restaurants have higher inspection scores on average than high risk restaurants. 

Since our test statistic was

$$\text{low risk mean} - \text{high risk mean}$$

larger values of the test statistic favor the alternative.

In [ ]:
...

This is lower than any cutoff we'd consider, so we'd reject the null hypothesis that the two groups of restaurants have similar inspection scores.

## Bakeries 🧁

<center>
<table><tr>
    <td><center><img src='data/pie.jpg'  style="width: 60%;"/></center></td>
    <td><center><img src='data/cupcakes.jpg'  style="width: 60%;"/></center></td>
</tr></table>
</center>

We'll load in a version of the restaurants dataset that has many more rows, some of which contain null values.

In [ ]:
restaurants_full = bpd.read_csv('data/restaurants_full.csv').get(keep_cols)
restaurants_full

Let's look at just the restaurants with `'Bake'` in the name that we know the inspection score for.

`.str.contains` can help us here.

In [ ]:
...

Some bakeries may have `'bake'` in their name, rather than `'Bake'`. To account for this, we can convert the entire Series to lowercase using `.str.lower()`, and then use `.str.contains('bake')`.

In [ ]:
...

In [ ]:
bakeries = ...
bakeries = ...
bakeries

We can plot the **population** distribution, i.e. the distribution of inspection scores for **all bakeries in San Francisco**.

In [ ]:
bakeries.plot(kind='hist', y='inspection_score', density=True, bins=score_bins, ec='w', figsize=(10, 5),
              title='Population Distribution');

For reference, the mean and standard deviation of the population distribution are calculated below.

In [ ]:
...

In this case we happen to have the inspection scores for all members of the population, but in reality we won't. So let's instead take a random **sample** of 200 bakeries from the population.

_Aside: Does the `.sample` method sample with or without replacement by default?_

In [ ]:
np.random.seed(23) # Ignore this

sample_of_bakeries = ...
sample_of_bakeries

We can plot the sample distribution:

In [ ]:
sample_of_bakeries.plot(kind='hist', y='inspection_score', density=True, bins=score_bins, ec='w', figsize=(10, 5),
                        title='Sample Distribution');

Note that since we took a large, random sample of the population, we expect that our sample looks similiar to the population and has a similar mean and SD.

In [ ]:
...

Indeed, the sample mean is quite close to the population mean, and the sample standard deviation is quite close to the population standard deviation.

Let's suppose we want to estimate the population mean (that is, the mean inspection score of all bakeries in SF).

One estimate of the population mean is the mean of our sample.

In [ ]:
sample_of_bakeries.get('inspection_score').mean()

However, our sample was random and could have been different, meaning our sample mean could also have been different.

**Question**: What's a reasonable range of possible values for the sample mean? **What is the distribution of the sample mean?**

### The Central Limit Theorem

> The Central Limit Theorem (CLT) says that the probability distribution of the **sum or mean** of a large random sample drawn with replacement will be roughly normal, regardless of the distribution of the population from which the sample is drawn.

In [ ]:
show_clt_slides()

To see an empirical distribution of the sample mean, let's take a large number of samples directly from the population and compute the mean of each one.

Remember, in real life we wouldn't be able to do this, since we wouldn't have access to the population.

In [ ]:
sample_means = np.array([])

...

In [ ]:
sample_means

In [ ]:
bpd.DataFrame().assign(sample_means=sample_means).plot(kind='hist', density=True, ec='w', bins=25, figsize=(10, 5));

Unsurprisingly, the distribution of the sample mean is bell-shaped. The CLT told us that!

The CLT also tells us that

$$\text{SD of Distribution of Possible Sample Means} = \frac{\text{Population SD}}{\sqrt{\text{sample size}}}$$



Let's try this out.

In [ ]:
np.std(bakeries.get('inspection_score')) / np.sqrt(200)

In [ ]:
np.std(sample_means)

Pretty close! Remember that `sample_means` is an array of simulated sample means; the more samples we simulate, the closer that `np.std(sample_means)` will get to the SD described by the CLT.

Note that in practice, we won't have the SD of the population, since we'll usually just have a single sample. In such cases, we can use the SD of the sample as an estimate of the SD of the population:

In [ ]:
np.std(sample_of_bakeries.get('inspection_score')) / np.sqrt(200)

Using the CLT, we have that the distribution of the sample mean:
- is roughly normal,
- is centered at the population mean (for which the sample mean is an estimate), and
- has a standard deviation of $\frac{\text{Population SD}}{\sqrt{\text{sample size}}}$ (which can be estimated using $\frac{\text{Sample SD}}{\sqrt{\text{sample size}}}$).

Using this information, we can build a confidence interval for where we think the population mean might be. A 95% confidence interval for the population mean is given by

$$
\left[
\text{sample mean} - 2\cdot \frac{\text{sample SD}}{\sqrt{\text{sample size}}}, \
\text{sample mean} + 2\cdot \frac{\text{sample SD}}{\sqrt{\text{sample size}}}
\right]
$$

In [ ]:
sample_mean = sample_of_bakeries.get('inspection_score').mean()
sample_std = np.std(sample_of_bakeries.get('inspection_score'))

In [ ]:
...

### Concept Check ✅ – Answer at [cc.dsc10.com](http://cc.dsc10.com) 

Using a single sample of 200 bakeries, how can we estimate the **median** inspection score of all bakeries in San Francisco with an inspection score? What technique should we use?

A. Standard hypothesis testing

B. Permutation testing  

C. Bootstrapping

D. The Central Limit Theorem

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
Bootstrapping. The CLT only applies to sample means (and sums), not to any other statistics.

</details>

There is no CLT for sample medians, so instead we'll have to resort to bootstrapping to estimate the distribution of the sample median.

Recall, bootstrapping is the act of **sampling from the original sample, with replacement**. This is also called **resampling**.

In [ ]:
# The median of our original sample – this is just one number
...

In [ ]:
# The median of a single bootstrap resample – this is just one number
...

Let's resample repeatedly.

In [ ]:
np.random.seed(23) # Ignore this

boot_medians = np.array([])

...

In [ ]:
boot_medians

In [ ]:
bpd.DataFrame().assign(boot_medians=boot_medians).plot(kind='hist', density=True, ec='w', bins=10, figsize=(10, 5));

Note that this distribution is not at all normal.

To compute a 95% confidence interval, we take the middle 95% of the bootstrapped medians.

In [ ]:
...

### Discussion Question

Which of the following interpretations of this confidence interval are valid?

1. 95% of SF bakeries have an inspection score between 85 and 88.  
2. 95% of the resamples have a median inspection score between 85 and 88.  
3. There is a 95% chance that our sample has a median inspection score between 85 and 88.  
4. There is a 95% chance that the median inspection score of all SF bakeries is between 85 and 88.  
5. If we had taken 100 samples from the same population, about 95 of these samples would have a median inspection score between 85 and 88.  
6.  If we had taken 100 samples from the same population, about 95 of the confidence intervals created would contain the median inspection score of all SF bakeries.  

<details>
<summary>Click for the answer <b>after</b> you've entered your guess above. <b>Don't scroll any further.</b></summary>
    
The correct answers are Option 2 and Option 6.

</details>

## Next time

### Next time

- One more review example.
- A high-level overview of the quarter.
- Some parting thoughts.